In [1]:
from fastai.vision.all import *
from fastai.vision.widgets import *import urllib.request


C:\Users\Artur\AppData\Local\Continuum\anaconda3\lib\site-packages\fastbook\__init__.py:22: UserWarning: Missing `graphviz` - please run `conda install fastbook`
  except ModuleNotFoundError: warn("Missing `graphviz` - please run `conda install fastbook`")


In [2]:
#This is the function that runs 
out_pl = widgets.Output()
lbl_pred = widgets.Label()
btn_run = widgets.Button(description='Classify')
btn_upload = widgets.FileUpload()

In [11]:

FILE_ID = "1_nhC0MjRmG3Y0vNydTAIrf5TqCsasxuO"
MODEL_URL = f"https://drive.google.com/uc?export=download&id={FILE_ID}"
urllib.request.urlopen(f"https://drive.google.com/open?id={FILE_ID}")
learn_inf = load_learner("export.pkl")

In [12]:
def on_click_classify(classify):
    img = PILImage.create(btn_upload.data[-1])
    out_pl.clear_output()
    with out_pl: display(img.to_thumb(128,128))
    pred,pred_idx,probs = learn_inf.predict(img)
    lbl_pred.value = f'Prediction: {pred}; Probability: {probs[pred_idx]:.04f}'

btn_run.on_click(on_click_classify) #Pass on_click_classify function to the on_click of our button

In [13]:
#hide_output
#VBox puts it all together in an array of buttons, labels, etc
VBox([widgets.Label('Select your bear!'), 
      btn_upload, btn_run, out_pl, lbl_pred])

In [ ]:
#https://mybinder.org/v2/gh/finnisvanilla/bear_viola_test/master?urlpath=%2Fvoila%2Frender%2FBears_Viola.ipynb